In [37]:
from gurobipy import *
import snap
import numpy as np
import copy
import random
import matplotlib.pyplot as plt
import time

In [38]:
#load a graph G
node = 781
# G = np.loadtxt('test_10nodes.txt', delimiter='\t', skiprows=3, dtype="int")
# G = np.loadtxt('facebook_combined.txt', delimiter=' ', dtype="int")
# G = G.tolist()

FIn = snap.TFIn("large_tree.graph")
G1 = snap.TNGraph.Load(FIn)

# select k nodes
k = 50

In [39]:
#build up internal opinion s
s = np.loadtxt('internal_opinion_large_tree.txt', dtype="float")
s = s.tolist()

In [40]:
W = [[0]*node for i in range(node)]
for NI in G1.Nodes():
    for Id in NI.GetOutEdges():
        W[NI.GetId()][Id]=1

V = []
for i in range(node):
    V.append([])

for i in range(len(W)):
    for j in range(len(W[i])):
        if W[i][j]==1:
            V[i].append(W[i].index(W[i][j]))
print(V)

[[], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [2], [2], [2], [2], [2], [3], [3], [3], [3], [3], [4], [4], [4], [4], [4], [5], [5], [5], [5], [5], [6], [6], [6], [6], [6], [7], [7], [7], [7], [7], [8], [8], [8], [8], [8], [9], [9], [9], [9], [9], [10], [10], [10], [10], [10], [11], [11], [11], [11], [11], [12], [12], [12], [12], [12], [13], [13], [13], [13], [13], [14], [14], [14], [14], [14], [15], [15], [15], [15], [15], [16], [16], [16], [16], [16], [17], [17], [17], [17], [17], [18], [18], [18], [18], [18], [19], [19], [19], [19], [19], [20], [20], [20], [20], [20], [21], [21], [21], [21], [21], [22], [22], [22], [22], [22], [23], [23], [23], [23], [23], [24], [24], [24], [24], [24], [25], [25], [25], [25], [25], [26], [26], [26], [26], [26], [27], [27], [27], [27], [27], [28], [28], [28], [28], [28], [29], [29], [29], [29], [29], [30], [30], [30], [30], [30], [31], [31], [31], [31], [31], [32], [32], [32], [32], [32], [33], [33], [33], [33], [33], [34], [34], [34], [34], [

In [41]:
#create blank model
MILP_model = Model('Mixed Integer Linear Program')

#add variables
Z = MILP_model.addVars([(i) for i in range(node)],lb=0, ub=1, name="Z")
Y = MILP_model.addVars([(i) for i in range(node)] ,vtype=GRB.BINARY, name="Y")
X = MILP_model.addVars([(i) for i in range(node)],lb=0, ub=1, name="X")
MILP_model.update()

#add objective number
MILP_model.setObjective(quicksum(Z[i] for i in range(node)), GRB.MAXIMIZE)

#constraint 1
MILP_model.addConstr(quicksum(Y[i] for i in range(node)) == k)
#constraint 2
MILP_model.addConstr(Z[0]==Y[0]+s[0]-s[0]*Y[0])
#constraint 3
MILP_model.addConstrs(Z[i] == Y[i]+(s[i]+Z[j])/2-(s[i]*Y[i])/2-X[i]/2 for i in range(1,node) for j in V[i])
#constraint 4
MILP_model.addConstrs(X[i]<=Z[j] for i in range(1,node) for j in V[i])

# MILP_model.addConstrs(Y[i]<=Z[i] for i in range(1,node))

MILP_model.update()
MILP_model.setParam("presolve",0)
MILP_model.setParam('SubMIPNodes',0)
MILP_model.setParam('Cuts',2)
MILP_model.setParam('MIRCuts',0)
MILP_model.setParam('Threads',1)

start = time.perf_counter()
MILP_model.optimize()
end = time.perf_counter()
print('time: ',end - start)

Changed value of parameter presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter SubMIPNodes to 0
   Prev: 500  Min: 0  Max: 2000000000  Default: 500
Changed value of parameter Cuts to 2
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter MIRCuts to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Threads to 1
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 1562 rows, 2343 columns and 5463 nonzeros
Model fingerprint: 0x61e4af82
Variable types: 1562 continuous, 781 integer (781 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-04, 5e+01]
Found heuristic solution: objective 387.0761773
Variable types: 1562 continuous, 781 integer (781 binary)

Root relaxation: objective 6

In [42]:
print(MILP_model.ObjVal)
for var in MILP_model.getVars():
    print(var.varName,'\t',var.x)

558.8729695901077
Z[0] 	 0.8463702418016275
Z[1] 	 0.8705573964763
Z[2] 	 1.0
Z[3] 	 1.0000000000000069
Z[4] 	 1.0
Z[5] 	 1.0
Z[6] 	 1.0
Z[7] 	 1.0
Z[8] 	 1.0
Z[9] 	 1.0
Z[10] 	 0.7367462537425962
Z[11] 	 1.0
Z[12] 	 0.9670529590539022
Z[13] 	 1.0
Z[14] 	 1.0
Z[15] 	 0.9383838400785574
Z[16] 	 0.8308544663847417
Z[17] 	 0.5829770373603378
Z[18] 	 1.0
Z[19] 	 0.9999999999999999
Z[20] 	 0.9767932923068061
Z[21] 	 0.8672582025631995
Z[22] 	 1.0
Z[23] 	 1.0
Z[24] 	 0.9538700751554854
Z[25] 	 0.8115200881452682
Z[26] 	 0.9311369225189956
Z[27] 	 0.8405839373709586
Z[28] 	 0.7902393868171391
Z[29] 	 0.9999999999999997
Z[30] 	 1.0
Z[31] 	 0.6710697771047368
Z[32] 	 0.7201610697652697
Z[33] 	 0.722210881020995
Z[34] 	 0.6954607105377958
Z[35] 	 0.9188778666808382
Z[36] 	 0.6281235569418341
Z[37] 	 0.6065547159418456
Z[38] 	 0.7267503328285527
Z[39] 	 0.7492566371454327
Z[40] 	 0.9852522378655086
Z[41] 	 0.7737501145272063
Z[42] 	 0.8931254631265136
Z[43] 	 1.0
Z[44] 	 0.6676481195322275
Z[45] 

Z[338] 	 0.9168969059616721
Z[339] 	 0.9165044533509601
Z[340] 	 0.8032777961309406
Z[341] 	 0.6811661169556493
Z[342] 	 0.7332820397364149
Z[343] 	 0.4153386529554341
Z[344] 	 0.4521563797414661
Z[345] 	 0.5254260831526287
Z[346] 	 0.7860045632643083
Z[347] 	 0.8311328649446674
Z[348] 	 0.5921560189048125
Z[349] 	 0.44078523667657743
Z[350] 	 0.42501422738820677
Z[351] 	 0.5923307705778416
Z[352] 	 0.5572125944585998
Z[353] 	 0.4957743716259376
Z[354] 	 0.8604962041826261
Z[355] 	 0.39047128138777093
Z[356] 	 0.7913700138690334
Z[357] 	 0.8992431861455371
Z[358] 	 0.9182147694979373
Z[359] 	 0.6747744045638658
Z[360] 	 0.6178895252714375
Z[361] 	 0.4678899870904532
Z[362] 	 0.7954748363002525
Z[363] 	 0.7522453140012302
Z[364] 	 0.457030112435265
Z[365] 	 0.41429092283569025
Z[366] 	 0.6237907921890846
Z[367] 	 0.7835159970118164
Z[368] 	 0.7621419573302938
Z[369] 	 0.9119757890015052
Z[370] 	 0.7880588606988038
Z[371] 	 0.4959246952150189
Z[372] 	 0.6742570155281675
Z[373] 	 0.802783

Z[707] 	 0.7441784751679177
Z[708] 	 0.8741784595606934
Z[709] 	 0.9425713896375338
Z[710] 	 0.8361586544134904
Z[711] 	 0.866140550593544
Z[712] 	 0.6951046323464316
Z[713] 	 0.5595606825869868
Z[714] 	 0.6591795828598882
Z[715] 	 0.7445338499311729
Z[716] 	 0.7357608150011727
Z[717] 	 0.8128799955736491
Z[718] 	 0.9832171849042914
Z[719] 	 0.9182972717036284
Z[720] 	 0.8892379803662218
Z[721] 	 0.6868159795038762
Z[722] 	 0.636218190285083
Z[723] 	 0.971921423303978
Z[724] 	 0.9140907947420718
Z[725] 	 0.8806987418083583
Z[726] 	 0.8396161444240231
Z[727] 	 0.551858006453904
Z[728] 	 0.7025843749740384
Z[729] 	 0.44032494070388983
Z[730] 	 0.498244807386434
Z[731] 	 0.7484210807594751
Z[732] 	 0.4095125814086628
Z[733] 	 0.6384350430791871
Z[734] 	 0.41947946655211105
Z[735] 	 0.4683109365112385
Z[736] 	 0.7082438798820555
Z[737] 	 0.4835320435096135
Z[738] 	 0.7953054708032925
Z[739] 	 0.5209379413026864
Z[740] 	 0.6249922536429219
Z[741] 	 0.882863050063871
Z[742] 	 0.5179863360784

Y[480] 	 -0.0
Y[481] 	 -0.0
Y[482] 	 -0.0
Y[483] 	 -0.0
Y[484] 	 -0.0
Y[485] 	 -0.0
Y[486] 	 -0.0
Y[487] 	 -0.0
Y[488] 	 -0.0
Y[489] 	 -0.0
Y[490] 	 -0.0
Y[491] 	 -0.0
Y[492] 	 -0.0
Y[493] 	 -0.0
Y[494] 	 -0.0
Y[495] 	 -0.0
Y[496] 	 -0.0
Y[497] 	 -0.0
Y[498] 	 -0.0
Y[499] 	 -0.0
Y[500] 	 -0.0
Y[501] 	 -0.0
Y[502] 	 -0.0
Y[503] 	 -0.0
Y[504] 	 -0.0
Y[505] 	 -0.0
Y[506] 	 -0.0
Y[507] 	 -0.0
Y[508] 	 -0.0
Y[509] 	 -0.0
Y[510] 	 -0.0
Y[511] 	 -0.0
Y[512] 	 -0.0
Y[513] 	 -0.0
Y[514] 	 -0.0
Y[515] 	 -0.0
Y[516] 	 -0.0
Y[517] 	 -0.0
Y[518] 	 -0.0
Y[519] 	 -0.0
Y[520] 	 -0.0
Y[521] 	 -0.0
Y[522] 	 -0.0
Y[523] 	 -0.0
Y[524] 	 -0.0
Y[525] 	 -0.0
Y[526] 	 -0.0
Y[527] 	 -0.0
Y[528] 	 -0.0
Y[529] 	 -0.0
Y[530] 	 -0.0
Y[531] 	 -0.0
Y[532] 	 -0.0
Y[533] 	 -0.0
Y[534] 	 -0.0
Y[535] 	 -0.0
Y[536] 	 -0.0
Y[537] 	 -0.0
Y[538] 	 -0.0
Y[539] 	 -0.0
Y[540] 	 -0.0
Y[541] 	 -0.0
Y[542] 	 -0.0
Y[543] 	 -0.0
Y[544] 	 -0.0
Y[545] 	 -0.0
Y[546] 	 -0.0
Y[547] 	 -0.0
Y[548] 	 -0.0
Y[549] 	 -0.0
Y[550] 	 -0.0
Y[551]

X[362] 	 0.0
X[363] 	 0.0
X[364] 	 0.0
X[365] 	 0.0
X[366] 	 0.0
X[367] 	 0.0
X[368] 	 0.0
X[369] 	 0.0
X[370] 	 0.0
X[371] 	 0.0
X[372] 	 0.0
X[373] 	 0.0
X[374] 	 0.0
X[375] 	 0.0
X[376] 	 0.0
X[377] 	 0.0
X[378] 	 0.0
X[379] 	 0.0
X[380] 	 0.0
X[381] 	 0.0
X[382] 	 0.0
X[383] 	 0.0
X[384] 	 0.0
X[385] 	 0.0
X[386] 	 0.0
X[387] 	 0.0
X[388] 	 0.0
X[389] 	 0.0
X[390] 	 0.0
X[391] 	 0.0
X[392] 	 0.0
X[393] 	 0.0
X[394] 	 0.0
X[395] 	 0.0
X[396] 	 0.0
X[397] 	 0.0
X[398] 	 0.0
X[399] 	 0.0
X[400] 	 0.0
X[401] 	 0.0
X[402] 	 0.0
X[403] 	 0.0
X[404] 	 0.0
X[405] 	 0.0
X[406] 	 0.0
X[407] 	 0.0
X[408] 	 0.0
X[409] 	 0.0
X[410] 	 0.0
X[411] 	 0.0
X[412] 	 0.0
X[413] 	 0.0
X[414] 	 0.0
X[415] 	 0.0
X[416] 	 0.0
X[417] 	 0.0
X[418] 	 0.0
X[419] 	 0.0
X[420] 	 0.0
X[421] 	 0.0
X[422] 	 0.0
X[423] 	 0.0
X[424] 	 0.0
X[425] 	 0.0
X[426] 	 0.0
X[427] 	 0.0
X[428] 	 0.0
X[429] 	 0.0
X[430] 	 0.0
X[431] 	 0.0
X[432] 	 0.0
X[433] 	 0.0
X[434] 	 0.0
X[435] 	 0.0
X[436] 	 0.0
X[437] 	 0.0
X[438] 	 0.0